# 결과 분석 시각화 코드

In [ ]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as img

import seaborn as sns
from pandas import DataFrame as df

# cv2 연습

In [1]:
img = cv2.imread('/opt/ml/input/data/batch_01_vt/0002.jpg', cv2.IMREAD_UNCHANGED)

cv2.rectangle(img, (100, 100), (200, 200), (255,250,0), 2)
cv2.putText(img, 'Text', (100, 100-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,250,0), 1)
 
images = [img]
titles = ['Test']
for i in range(len(images)):
    plt.figure(figsize=(10,10))
    plt.subplot(1, len(images), i+1),
    plt.title(titles[i]),
    plt.imshow(images[i]),
    plt.xticks([]), 
    plt.yticks([]),
plt.show()


In [2]:
import pandas as pd

root_dir = '/opt/ml/output/'
data1_file = 'best.csv'
data2_file = 'best_mosaic.csv'

data1 = pd.read_csv(root_dir + data1_file)
data2 = pd.read_csv(root_dir + data2_file)

data2.head()

# one line box 분포도 구하기

In [105]:
total_boxes_info1 = [] # 이미지별 - > class, x1, y1, x2, y2, confidence
total_boxes_info2 = [] # 이미지별 - > class, x1, y1, x2, y2, confidence


for i in range(len(data1)):
    prediction_string = data1.iloc[i]['PredictionString']
    
    boxes_info = [] # class, x1, y1, x2, y2, confidence
     
    prediction_info = prediction_string.split()
    
    for j in range(0,len(prediction_info),6):
        class_info = int(prediction_info[j])
        x1 = float(prediction_info[j+2])
        y1 = float(prediction_info[j+3])
        x2 = float(prediction_info[j+4])
        y2 = float(prediction_info[j+5])
        prediction = float(prediction_info[j+1])
        
        boxes_info.append([class_info,x1,y1,x2,y2, prediction])
    total_boxes_info1.append(boxes_info)
    
for i in range(len(data2)):
    prediction_string = data2.iloc[i]['PredictionString']
    
    if not isinstance(prediction_string,str):
        continue
    
    boxes_info = [] # class, x1, y1, x2, y2, confidence
    
    prediction_info = prediction_string.split()
    
    for j in range(0,len(prediction_info),6):
        class_info = int(prediction_info[j])
        x1 = float(prediction_info[j+2])
        y1 = float(prediction_info[j+3])
        x2 = float(prediction_info[j+4])
        y2 = float(prediction_info[j+5])
        prediction = float(prediction_info[j+1])
        
        boxes_info.append([class_info,x1,y1,x2,y2, prediction])
    total_boxes_info2.append(boxes_info)

In [106]:
category_list = ['UNKNOWN',
 'General trash',
 'Paper',
 'Paper pack',
 'Metal',
 'Glass',
 'Plastic',
 'Styrofoam',
 'Plastic bag',
 'Battery',
 'Clothing']

count_list1 = [0,0,0,0,0,0,0,0,0,0,0]
count_list2 = [0,0,0,0,0,0,0,0,0,0,0]

for total_box_info in total_boxes_info1:
    for info in total_box_info:
        class_info = info[0]
        x1 = info[1]
        y1 = info[2]
        x2 = info[3]
        y2 = info[4]
        confidence = info[5]
        
        if x1 == x2 or y1 == y2:
            count_list1[class_info] += 1
            
for total_box_info in total_boxes_info2:
    for info in total_box_info:
        class_info = info[0]
        x1 = info[1]
        y1 = info[2]
        x2 = info[3]
        y2 = info[4]
        confidence = info[5]
        
        if x1 == x2 or y1 == y2:
            count_list2[class_info] += 1

In [107]:
line_box_stat1 = df(data= {"category" : category_list, "linebox_count" : count_list1},
               columns = ["category", "linebox_count"]
              )
line_box_stat2 = df(data= {"category" : category_list, "linebox_count" : count_list2},
               columns = ["category", "linebox_count"]
              )

In [3]:
# 카테고리별 line bbox개수 분포도
fig, axes = plt.subplots(1, 2, figsize = (30, 6))

fig.suptitle("line box Distribution", fontsize=30)
axes[0].set_title(data1_file,fontdict={'fontsize' : 20})
axes[1].set_title(data2_file,fontdict={'fontsize' : 20})
sns.barplot(x = "category", y = "linebox_count",data = line_box_stat1, ax = axes[0])
sns.barplot(x = "category", y = "linebox_count",data = line_box_stat2, ax = axes[1])
print("line box 개수 : ", sum(count_list1))
print("line box개수" , sum(count_list2))

# One Line 클래스별 Confidence구하기

In [109]:
average_confidence_one_line_boxes1 = [0,0,0,0,0,0,0,0,0,0,0]
average_confidence_one_line_boxes_count1 = [0,0,0,0,0,0,0,0,0,0,0]

for total_box_info in total_boxes_info1:
    for info in total_box_info:
        class_info = info[0]
        x1 = info[1]
        y1 = info[2]
        x2 = info[3]
        y2 = info[4]
        confidence = info[5]
        
        if x1 == x2 or y1 == y2:
            average_confidence_one_line_boxes1[class_info] += confidence
            average_confidence_one_line_boxes_count1[class_info] += 1

average_confidence_one_line_boxes2 = [0,0,0,0,0,0,0,0,0,0,0]
average_confidence_one_line_boxes_count2 = [0,0,0,0,0,0,0,0,0,0,0]

for total_box_info in total_boxes_info2:
    for info in total_box_info:
        class_info = info[0]
        x1 = info[1]
        y1 = info[2]
        x2 = info[3]
        y2 = info[4]
        confidence = info[5]
        
        if x1 == x2 or y1 == y2:
            average_confidence_one_line_boxes2[class_info] += confidence
            average_confidence_one_line_boxes_count2[class_info] += 1

In [110]:
# confidence값 평균
if sum(average_confidence_one_line_boxes_count1) != 0:
    print(sum(average_confidence_one_line_boxes1) / sum(average_confidence_one_line_boxes_count1))
    
if sum(average_confidence_one_line_boxes_count2) != 0:
    print(sum(average_confidence_one_line_boxes2) / sum(average_confidence_one_line_boxes_count2))

0.004779275566631141


In [111]:
for i in range(11):
    if average_confidence_one_line_boxes_count1[i] != 0:
        average_confidence_one_line_boxes1[i] /= average_confidence_one_line_boxes_count1[i]
        
for i in range(11):
    if average_confidence_one_line_boxes_count2[i] != 0:
        average_confidence_one_line_boxes2[i] /= average_confidence_one_line_boxes_count2[i]

In [4]:
box_stat1 = df(data= {"category" : category_list, "linebox_confidence_average" : average_confidence_one_line_boxes1},
               columns = ["category", "linebox_confidence_average"]
              )
box_stat2 = df(data= {"category" : category_list, "linebox_confidence_average" : average_confidence_one_line_boxes2},
               columns = ["category", "linebox_confidence_average"]
              )

fig, axes = plt.subplots(1, 2, figsize = (30, 6))

fig.suptitle("One line box confidence value", fontsize=30)
axes[0].set_title(data1_file,fontdict={'fontsize' : 20})
axes[1].set_title(data2_file,fontdict={'fontsize' : 20})
sns.barplot(x = "category", y = "linebox_confidence_average",data = box_stat1, ax = axes[0])
sns.barplot(x = "category", y = "linebox_confidence_average",data = box_stat2, ax = axes[1])

## 클래스별 박스 예측 분포도 분석

In [5]:
# 클래스별 박스 예측 분포

boxes_category_count1 = [0,0,0,0,0,0,0,0,0,0,0]
boxes_category_count2 = [0,0,0,0,0,0,0,0,0,0,0]

for total_box_info in total_boxes_info1:
    for info in total_box_info:
        class_info = info[0]
        x1 = info[1]
        y1 = info[2]
        x2 = info[3]
        y2 = info[4]
        confidence = info[5]
        
        boxes_category_count1[class_info] += 1
        
for total_box_info in total_boxes_info2:
    for info in total_box_info:
        class_info = info[0]
        x1 = info[1]
        y1 = info[2]
        x2 = info[3]
        y2 = info[4]
        confidence = info[5]
        
        boxes_category_count2[class_info] += 1
        
box_stat1 = df(data= {"category" : category_list, "boxes_category_count" :boxes_category_count1},
               columns = ["category", "boxes_category_count"]
              )
box_stat2 = df(data= {"category" : category_list, "boxes_category_count" :boxes_category_count2},
               columns = ["category", "boxes_category_count"]
              )

fig, axes = plt.subplots(1, 2, figsize = (30, 6))

axes[0].set_title(data1_file,fontdict={'fontsize' : 20})
axes[1].set_title(data2_file,fontdict={'fontsize' : 20})

fig.suptitle("boxes distribution by class", fontsize=30)
sns.barplot(x = "category", y = "boxes_category_count",data = box_stat1, ax = axes[0])
sns.barplot(x = "category", y = "boxes_category_count",data = box_stat2, ax = axes[1])

print(box_stat1)
print(box_stat2)

## train데이터의 class별 bbox분포 -> 예측과 분포도가 비슷

In [6]:
# 카테고리별 bbox개수 분포도
fig, ax1 = plt.subplots(1, 1, fcigsize = (30, 6))

sns.barplot(x = "category", y = "count",data = bbox_stat, ax = ax1)

In [7]:
# confidence 분포
boxes_confidence1 = [0,0,0,0,0,0,0,0,0,0,0]
boxes_confidence_count1 = [0,0,0,0,0,0,0,0,0,0,0]

boxes_confidence2 = [0,0,0,0,0,0,0,0,0,0,0]
boxes_confidence_count2 = [0,0,0,0,0,0,0,0,0,0,0]

for total_box_info in total_boxes_info1:
    for info in total_box_info:
        class_info = info[0]
        x1 = info[1]
        y1 = info[2]
        x2 = info[3]
        y2 = info[4]
        confidence = info[5]
        
        boxes_confidence1[class_info] += confidence
        boxes_confidence_count1[class_info] += 1
        
for i in range(11):
    boxes_confidence1[i] /= boxes_confidence_count1[i]
    
for total_box_info in total_boxes_info2:
    for info in total_box_info:
        class_info = info[0]
        x1 = info[1]
        y1 = info[2]
        x2 = info[3]
        y2 = info[4]
        confidence = info[5]
        
        boxes_confidence2[class_info] += confidence
        boxes_confidence_count2[class_info] += 1
        
for i in range(11):
    boxes_confidence2[i] /= boxes_confidence_count2[i]
        
box_stat1 = df(data= {"category" : category_list, "boxes_confidence" :boxes_confidence1},
               columns = ["category", "boxes_confidence"]
              )

box_stat2= df(data= {"category" : category_list, "boxes_confidence" :boxes_confidence2},
               columns = ["category", "boxes_confidence"]
              )

fig, axes = plt.subplots(1, 2, figsize = (30, 6))

fig.suptitle("confidence distribution by class", fontsize=30)
axes[0].set_title(data1_file,fontdict={'fontsize' : 20})
axes[1].set_title(data2_file,fontdict={'fontsize' : 20})
sns.barplot(x = "category", y = "boxes_confidence",data = box_stat1, ax = axes[0])
sns.barplot(x = "category", y = "boxes_confidence",data = box_stat2, ax = axes[1])

# 결과에 대한 S, M, L박스의 분포도

In [8]:
total_boxes_info1[0]

In [9]:
# 총 S, M, L박스의 inference갯수 비교
box_scale_count1 = [0,0,0]
box_scale_count2 = [0,0,0]
scale_categories = ["small", "medium","large"]

for total_box_info in total_boxes_info1:
    for i in range(len(total_box_info)):
        info = total_box_info[i]
        w = info[3]  - info[1]
        h = info[4] - info[2]
        area = w * h
        if area < 32* 32:
            box_scale_count1[0] += 1
        elif 32 * 32 <= area < 96 * 96 :
            box_scale_count1[1] += 1
        else:
            box_scale_count1[2] += 1

for total_box_info in total_boxes_info2:
    for i in range(len(total_box_info)):
        info = total_box_info[i]
        w = info[3]  - info[1]
        h = info[4] - info[2]
        area = w * h
        if area < 32* 32:
            box_scale_count2[0] += 1
        elif 32 * 32 <= area < 96 * 96 :
            box_scale_count2[1] += 1
        else:
            box_scale_count2[2] += 1


box_stat1 = df(data= {"scale category" : scale_categories, "count" : box_scale_count1},
               columns = ["scale category", "count"]
              )

box_stat2= df(data= {"scale category" : scale_categories, "count" : box_scale_count2},
               columns = ["scale category", "count"]
              )

fig, axes = plt.subplots(1, 2, figsize = (30, 6))

fig.suptitle("boxes category distribution", fontsize=30)
axes[0].set_title(data1_file,fontdict={'fontsize' : 20})
axes[1].set_title(data2_file,fontdict={'fontsize' : 20})
sns.barplot(x = "scale category", y = "count",data = box_stat1, ax = axes[0])
sns.barplot(x = "scale category", y = "count",data = box_stat2, ax = axes[1])

## 클래스별 small, mediumm large박스 분포

In [10]:
# 클래스별 small, mediumm large박스 분포
class_category_list1 = []
box_category_list1 = []

class_category_list2 = []
box_category_list2 = []

class_categories = ['UNKNOWN','General trash','Paper','Paper pack','Metal','Glass',
                  'Plastic','Styrofoam','Plastic bag','Battery','Clothing']

scale_categories = ["small", "medium","large"]
for total_box_info in total_boxes_info1:
    for i in range(len(total_box_info)):
        info = total_box_info[i]
        class_category_list1.append(class_categories[info[0]])
        w = info[3]  - info[1]
        h = info[4] - info[2]
        area = w * h
        if area < 32* 32:
            box_category_list1.append(scale_categories[0])
        elif 32 * 32 <= area < 96 * 96 :
            box_category_list1.append(scale_categories[1])
        else:
            box_category_list1.append(scale_categories[2])
            
for total_box_info in total_boxes_info2:
    for i in range(len(total_box_info)):
        info = total_box_info[i]
        class_category_list2.append(class_categories[info[0]])
        w = info[3]  - info[1]
        h = info[4] - info[2]
        area = w * h
        if area < 32* 32:
            box_category_list2.append(scale_categories[0])
        elif 32 * 32 <= area < 96 * 96 :
            box_category_list2.append(scale_categories[1])
        else:
            box_category_list2.append(scale_categories[2])
            
        
            
box_stat1 = df(data= {"class category" : class_category_list1, "box category" : box_category_list1},
               columns = ["class category", "box category"]
              )
            
box_stat2 = df(data= {"class category" : class_category_list2, "box category" : box_category_list2},
               columns = ["class category", "box category"]
              )

fig, axes = plt.subplots(2, 1, figsize = (30, 15))

fig.suptitle("boxes category distribution by class", fontsize=30)
axes[0].set_title(data1_file,fontdict={'fontsize' : 20})
axes[1].set_title(data2_file,fontdict={'fontsize' : 20})
sns.countplot(x = "class category", hue = "box category",data = box_stat1, ax = axes[0], order = class_categories)
sns.countplot(x = "class category", hue = "box category",data = box_stat2, ax = axes[1], order = class_categories)


# validation데이터를 통한 모델의 분석

- True positive 관점


- False Positive관점


- False Negative관점

In [96]:
root_dir = '/opt/ml/validation_output/'
data_file = 'swin_val_oversampling_check.csv'

data = pd.read_csv(root_dir + data_file)

total_boxes_info = {}

for i in range(len(data)):
    if not isinstance(data.iloc[i]['PredictionString'], str):
        total_boxes_info[image_file] = []
        continue
    
    prediction_string = data.iloc[i]['PredictionString']
    image_file = data.iloc[i]['image_id']
    
    boxes_info = [] # class, x1, y1, x2, y2, confidence
     
    prediction_info = prediction_string.split()
    
    for j in range(0,len(prediction_info),6):
        class_info = int(prediction_info[j])
        x1 = float(prediction_info[j+2])
        y1 = float(prediction_info[j+3])
        x2 = float(prediction_info[j+4])
        y2 = float(prediction_info[j+5])
        prediction = float(prediction_info[j+1])
        
        boxes_info.append([class_info,x1,y1,x2,y2, prediction])
    total_boxes_info[image_file] = boxes_info


In [97]:
def get_iou(pred_box, gt_box):
    """iou 계산 함수
    """
    # 1.get the coordinate of inters
    ixmin = max(pred_box[0], gt_box[0])
    ixmax = min(pred_box[2], gt_box[2])
    iymin = max(pred_box[1], gt_box[1])
    iymax = min(pred_box[3], gt_box[3])

    iw = np.maximum(ixmax-ixmin+1., 0.)
    ih = np.maximum(iymax-iymin+1., 0.)

    # 2. calculate the area of inters
    inters = iw*ih

    # 3. calculate the area of union
    uni = ((pred_box[2]-pred_box[0]+1.) * (pred_box[3]-pred_box[1]+1.) +
           (gt_box[2] - gt_box[0] + 1.) * (gt_box[3] - gt_box[1] + 1.) -
           inters)

    # 4. calculate the overlaps between pred_box and gt_box
    iou = inters / uni
    
    return iou

In [11]:
from pycocotools.coco import COCO

coco = COCO('/opt/ml/input/data/val.json')
image_ids = coco.getImgIds()

In [99]:
TP_class_count = [0,0,0,0,0,0,0,0,0,0,0]
FP_class_count = [0,0,0,0,0,0,0,0,0,0,0]
FN_class_count = [0,0,0,0,0,0,0,0,0,0,0]

class_total_count = [0,0,0,0,0,0,0,0,0,0,0]


for i in image_ids:
    # 이미지에 대한 어노테이션 가져오기
    image_infos = coco.loadImgs(i)[0]
    file_name = image_infos['file_name']
    
    if file_name not in total_boxes_info.keys():
        continue
    
    inference_infos = total_boxes_info[file_name]
    
    ground_truth_infos = []
    ann_ids = coco.getAnnIds(imgIds = i)
    anns = coco.loadAnns(ann_ids)
    for ann in anns:
        class_info = ann['category_id']
        box_info = ann['bbox']
        x1 = box_info[0]
        y1 = box_info[1]
        x2 = x1 + box_info[2]
        y2 = y1 + box_info[3]
        ground_truth_infos.append([class_info, x1,y1, x2, y2])
    
    
    eval_box_class_check = set()
    
    # gt에서 이 inference는 몇개나 맞추는지 확인
    for ground_truth_info in ground_truth_infos:
        class_total_count[ground_truth_info[0]] += 1
    
    for inference_info in inference_infos:
        for gt_info_idx, ground_truth_info in enumerate(ground_truth_infos):
            if inference_info[0] != ground_truth_info[0]:
                continue
            # 같다면 iou를 계산
            iou_score = get_iou([inference_info[1],inference_info[2],inference_info[3],inference_info[4]],
                               [ground_truth_info[1],ground_truth_info[2], ground_truth_info[3],ground_truth_info[4]])
            
            if inference_info[0] == ground_truth_info[0]:
                if iou_score >= 0.5:
                    TP_class_count[inference_info[0]] += 1 # 옳은 검출(iou 0.5이상)
                    eval_box_class_check.add(gt_info_idx)
                else:
                    FP_class_count[inference_info[0]] += 1 # 틀린 검출(iou 0.5 미만)
#             else:
#                 FP_class_count[inference_info[0]] += 1
    
    for i in range(len(ground_truth_infos)):
        if i not in eval_box_class_check:
            FN_class_count[ground_truth_infos[i][0]] += 1 # 검출 되어야 할 것이 검출되지 않음
        
    

for i in range(11):
    TP_class_count[i] = TP_class_count[i] / class_total_count[i]
    FP_class_count[i] = FP_class_count[i] / class_total_count[i]
    FN_class_count[i] = FN_class_count[i] / class_total_count[i]
    

In [12]:
print(TP_class_count)
print(FP_class_count)
print(FN_class_count)

In [13]:
class_categories = ['UNKNOWN','General trash','Paper','Paper pack','Metal','Glass',
                  'Plastic','Styrofoam','Plastic bag','Battery','Clothing']

box_stat = df(data= {"class category" : class_categories, "value" : TP_class_count},
               columns = ["class category", "value"]
              )

fig, axes = plt.subplots(1, 1, figsize = (30, 15))

fig.suptitle("True Positive Aspect", fontsize=30)
axes.set_title(data_file,fontdict={'fontsize' : 20})
sns.barplot(x = "class category", y = "value",data = box_stat, ax = axes)

In [14]:
class_categories = ['UNKNOWN','General trash','Paper','Paper pack','Metal','Glass',
                  'Plastic','Styrofoam','Plastic bag','Battery','Clothing']

box_stat = df(data= {"class category" : class_categories, "value" : FP_class_count},
               columns = ["class category", "value"]
              )

fig, axes = plt.subplots(1, 1, figsize = (30, 15))

fig.suptitle("False Positive Aspect", fontsize=30)
axes.set_title(data_file,fontdict={'fontsize' : 20})
sns.barplot(x = "class category", y = "value",data = box_stat, ax = axes)

In [15]:
class_categories = ['UNKNOWN','General trash','Paper','Paper pack','Metal','Glass',
                  'Plastic','Styrofoam','Plastic bag','Battery','Clothing']

box_stat = df(data= {"class category" : class_categories, "value" : FN_class_count},
               columns = ["class category", "value"]
              )

fig, axes = plt.subplots(1, 1, figsize = (30, 15))

fig.suptitle("False Negative Aspect", fontsize=30)
axes.set_title(data_file,fontdict={'fontsize' : 20})
sns.barplot(x = "class category", y = "value",data = box_stat, ax = axes)

## 모델 예측 결과 시각화

In [56]:
import matplotlib.image as img
def print_result(image_path, boxes_info):
    image = img.imread(image_path)
    
    plt.figure(figsize=(12,17))
    plt.axis('off')

    plt.imshow(image)
    
    _class_names= [ "UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing" ]
    
    _class_colors = ['black', 'brown', 'orange', 'pink', 'green',
                     'gray', 'red', 'olive', 'purple', 'cyan',
                     'blue', 'seagreen'
                    ]
    
    ax = plt.gca()
    
    
    for i in range(len(boxes_info)):        
        rect = patches.Rectangle((boxes_info[i][1],boxes_info[i][2]),
                                 boxes_info[i][3] - boxes_info[i][1],
                                 boxes_info[i][4] - boxes_info[i][2],
                                 linewidth=10,
                                 edgecolor=_class_colors[boxes_info[i][0]],
                                 fill = False)
        ax.add_patch(rect)
    
    plt.show()

In [16]:
data1.head()

In [17]:
# Normal결과 확인해보기
root_dir = "/opt/ml/input/data/"
data_vis_file = "jin.csv"
data_vis = pd.read_csv('/opt/ml/output/jin.csv')

temp = []
for i in range(213,214):
    image_path = root_dir + data_vis.iloc[i]['image_id']
    prediction_string = data_vis.iloc[i]['PredictionString']
    
    prediction_info = prediction_string.split()
    
    boxes_info = [] # class, x1, y1, x2, y2, confidence
    
    for j in range(0,len(prediction_info),6):
        class_info = int(prediction_info[j])
        x1 = float(prediction_info[j+2])
        y1 = float(prediction_info[j+3])
        x2 = float(prediction_info[j+4])
        y2 = float(prediction_info[j+5])
        prediction = float(prediction_info[j+1])
        
        
        boxes_info.append([class_info,x1,y1,x2,y2, prediction])
    
    # 이미지 시각화 과정
    
    num = print_result(image_path, boxes_info)
    
    temp = boxes_info
    

# 70, 213, 332, 346. 368, 467, 506...

In [18]:
root_dir = "/opt/ml/input/data/"
for i in range(70,71):
    image_path = root_dir + data2.iloc[i]['image_id']
    prediction_string = data2.iloc[i]['PredictionString']
    
    prediction_info = prediction_string.split()
    
    boxes_info = [] # class, x1, y1, x2, y2, confidence
    
    for i in range(0,len(prediction_info),6):
        class_info = int(prediction_info[i])
        x1 = float(prediction_info[i+2])
        y1 = float(prediction_info[i+3])
        x2 = float(prediction_info[i+4])
        y2 = float(prediction_info[i+5])
        prediction = float(prediction_info[i+1])
        
        boxes_info.append([class_info,x1,y1,x2,y2, prediction])
    
    # 이미지 시각화 과정
    
    print_result(image_path, boxes_info)

In [19]:
root_dir = "/opt/ml/input/data/"
for i in range(0,10):
    image_path = root_dir + data3.iloc[i]['image_id']
    prediction_string = data3.iloc[i]['PredictionString']
    
    prediction_info = prediction_string.split()
    
    boxes_info = [] # class, x1, y1, x2, y2, confidence
    
    for i in range(0,len(prediction_info),6):
        class_info = int(prediction_info[i])
        x1 = float(prediction_info[i+2])
        y1 = float(prediction_info[i+3])
        x2 = float(prediction_info[i+4])
        y2 = float(prediction_info[i+5])
        prediction = float(prediction_info[i+1])
        
        boxes_info.append([class_info,x1,y1,x2,y2, prediction])
    
    # 이미지 시각화 과정
    
    print_result(image_path, boxes_info)

In [189]:
data = pd.read_csv('/opt/ml/output/my.csv')

submission = pd.DataFrame()



In [20]:
import os

prediction_strings = []
file_names = []

for i in range(len(data)):
    prediction_string = data1.iloc[i]['PredictionString']
    file_name = data1.iloc[i]['image_id']
    
    prediction_info = prediction_string.split()
    
    new_prediction_string = ""
    
    for j in range(0,len(prediction_info),6):
        class_info = int(prediction_info[j])
        x1 = float(prediction_info[j+2])
        y1 = float(prediction_info[j+3])
        x2 = float(prediction_info[j+4])
        y2 = float(prediction_info[j+5])
        prediction = float(prediction_info[j+1])
        
        if x1 != x2 and y1 != y2:
            new_prediction_string += str(class_info) + ' ' + str(prediction) + ' ' + str(x1) + ' ' + str(y1) + ' ' + str(x2) + ' ' + str(y2) + ' '
    prediction_strings.append(new_prediction_string)
    file_names.append(file_name)
    
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join('./', f'submission_check.csv'), index=None)
submission.head()